<a href="https://colab.research.google.com/github/dano1234/SharedMindsF23/blob/master/Week3/ConnectToLlama2ColabViaNGrok/Notebook/Shared_Minds_Web_to_LLaMa_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Share Minds
This is based on [this Notebook ](https://colab.research.google.com/drive/1SQmK0GYz34RGVlOnL5YMkdm7hXD6OjQT?usp=sharing#scrollTo=kzG2YoH4dil_)and [this video](https://www.youtube.com/watch?v=Z6sCl6abJj4) by Kris Ograbek

## Introduction
In this Colab Notebook, we are going to explore Llama-2 7B, a model fine-tuned for generating text & chatting.

By the end of this tutorial, you'll be able to interact with this model and use it to generate conversational responses.

Whether you're curious about chatbot technology or simply want to see a machine-generated response to a particular question, this notebook will serve as a comprehensive guide.

## Workflow
1. **Installations**: We'll begin by setting up our environment with the required libraries.
2. **Prerequisites**: Ensure we have access to the Llama-2 7B model on Hugging Face.
3. **Loading the Model & Tokenizer**: Retrieve the model and tokenizer for our session.
4. **Creating the Llama Pipeline**: Prepare our model for generating responses.
5. **Interacting with Llama**: Prompt the model for answers and explore its capabilities.

Let's dive in!

**First, change runtime to GPU.**


You can play with Llama-2 7B Chat here: https://huggingface.co/spaces/huggingface-projects/llama-2-7b-chat

## Installations

Before we proceed, we need to ensure that the essential libraries are installed:
- `Hugging Face Transformers`: Provides us with a straightforward way to use pre-trained models.
- `PyTorch`: Serves as the backbone for deep learning operations.
- `Accelerate`: Optimizes PyTorch operations, especially on GPU.

In [1]:
!pip install transformers torch accelerate
#install web libraries
!pip install fastapi nest-asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=b522bd55fcada92c77b470930451a5ec11582e010e537ff0f158bd87460d5f8b
  Store

### Prerequisites

To load our desired model, `meta-llama/Llama-2-7b-chat-hf`, we first need to authenticate ourselves on Hugging Face. This ensures we have the correct permissions to fetch the model.

1. Gain access to the model on Hugging Face: [Link](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).  DON'T FOGET TO DO THIS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
2. Use the Hugging Face CLI to login and verify your authentication status.



In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
!huggingface-cli whoami

dano1234
orgs:  itpima,sd-diffusers-pipelines-library


### Loading Model & Tokenizer

Here, we are preparing our session by loading both the Llama model and its associated tokenizer.

The tokenizer will help in converting our text prompts into a format that the model can understand and process.

In [5]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-hf

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


### Creating the Llama Pipeline

We'll set up a pipeline for text generation.

This pipeline simplifies the process of feeding prompts to our model and receiving generated text as output.

*Note*: This cell takes 2-3 minutes to run

In [7]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Getting Responses

With everything set up, let's see how Llama responds to some sample queries.

In [8]:
def get_llama_response(prompt):
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=50,
    )
    print("Chatbot:", sequences[0]['generated_text'])

    result = get_llama_response(prompt)
    return result

In [ ]:
result =  get_llama_response("how deep is your love")
result

Set up the web server

In [ ]:
from fastapi import FastAPI
import nest_asyncio
import json
from pyngrok import ngrok
import uvicorn
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from fastapi import Request, FastAPI



#this is another way to package the incoming stuff so the documentation will see the structure.  We are not using
class DataIn(BaseModel):
    data: str

app = FastAPI()

origins = [
    "*",
    "http://localhost",
    "http://localhost:8080",
    "http://127.0.0.1:8000",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"]
)


@app.get("/")
async def who():
  return {"Who": "Are You"}

@app.get("/hello/{name}")
async def hello(name):

  return {"Hello": name}


#@app.post.get("/helloBaseModel/")
#async def create_item(dataIn: DataIn):
#  return {"Hello": dataIn.prompt}


@app.post("/generateIt/")
async def get_body(request: Request):
  data = await request.json()
  promptFromClient = data["input"]["prompt"];
  output = get_llama_response(promptFromClient)
  d = {
    "prompt": promptFromClient,
    "response": output,
  }
  #return jsonify(d)
  json_compatible_item_data = jsonable_encoder(d)
  return JSONResponse(content=json_compatible_item_data)

ngrok.kill()
#ngrok_tunnel = ngrok.connect(8000) # if you don't sign up for a domain you can do this.
#####OR
#PUT SIGN UP WITH NGROK AND GET A PERMANENT DOMAIN. PUT YOUR OWN STUFF IN HERE Gz
ngrok.set_auth_token("2IHXCSXJHnT0NVg0iQbojzGqfcR_3chN1kK4rE2eFi3Ezd4")
ngrok_tunnel = ngrok.connect(8000, "http", domain="dano.ngrok.dev")

print('Public URL to copy in your js code:', ngrok_tunnel.public_url)
nest_asyncio.apply()


uvicorn.run(app, port=8000)


INFO:     Started server process [15607]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL to copy in your js code: https://dano.ngrok.dev
INFO:     24.161.69.19:0 - "OPTIONS /generateIt/ HTTP/1.1" 200 OK
Chatbot: A student trying to learn how use a machine learning API for the first time.

### Prerequisites

* Familiarity with Python programming language
* Basic understanding of machine learning concepts (e.g. supervised learning
Chatbot: A student trying to learn how use a machine learning API for the first time.

### Prerequisites

* Familiarity with Python programming language
* Basic understanding of machine learning concepts (e.g. supervised learning
Chatbot: A student trying to learn how use a machine learning API for the first time.

### Prerequisites

* Familiarity with Python programming language
* Basic understanding of machine learning concepts (e.g. supervised learning
Chatbot: A student trying to learn how use a machine learning API for the first time.

### Prerequisites

* Familiarity with Python programming language
* Basic understanding of machine

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Chatbot: A student trying to learn how use a machine learning API for the first time.

### Prerequisites

* Familiarity with Python programming language
* Basic understanding of machine learning concepts (e.g. supervised learning
